<a href="https://colab.research.google.com/github/aakashbansalgit/python/blob/master/Explicit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Explicit

In [3]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
from datetime import timedelta

In [4]:
# ENTER INPUT FOR: start_step
pricing_date = '2024-12-09'
last_obs = '2026-01-09'
issue_date = date(2024, 12, 9)


# Calculate time to maturity in years
start_date = datetime.strptime(pricing_date, '%Y-%m-%d')
end_date = datetime.strptime(last_obs, '%Y-%m-%d')
T = (end_date - start_date).days / 365.0

jmax = 250 #Number of S steps
imax = 396 * 20 #Number of t steps
S0 = 246.75 # Initial underlying price
K = 192.465 # Coupon barrier and Knock-in value
sigma = 0.236 # Volatility at 100% moneyness
r = 0.04334 # Between Pricing date and Final Valuation date
q = 0.00378
SL = 0 #Minimum S value
SU = 2.5 * S0 #Maximum S value
coupon = 7.5 # Coupon payment

# Calculate step sizes
dt = T / imax
dS = SU / jmax

# Define valuation dates
valuation_dates = [
    date(2025, 1, 9),   # January 9, 2025
    date(2025, 2, 10),  # February 10, 2025
    date(2025, 3, 10),  # March 10, 2025
    date(2025, 4, 9),   # April 9, 2025
    date(2025, 5, 9),   # May 9, 2025
    date(2025, 6, 9),   # June 9, 2025
    date(2025, 7, 9),   # July 9, 2025
    date(2025, 8, 11),  # August 11, 2025
    date(2025, 9, 9),   # September 9, 2025
    date(2025, 10, 9),  # October 9, 2025
    date(2025, 11, 10), # November 10, 2025
    date(2025, 12, 9),  # December 9, 2025
    date(2026, 1, 9)    # January 9, 2026
]

# Convert to year fractions
val_times = np.array([
    (date - issue_date).days / 365.0 for date in valuation_dates
])

# Convert to timesteps
val_timesteps = (val_times / dt).astype(int)

# Define autocall dates
autocall_dates = [
    date(2025, 6, 9),   # June 9, 2025
    date(2025, 7, 9),   # July 9, 2025
    date(2025, 8, 11),  # August 11, 2025
    date(2025, 9, 9),   # September 9, 2025
    date(2025, 10, 9),  # October 9, 2025
    date(2025, 11, 10), # November 10, 2025
    date(2025, 12, 9)   # December 9, 2025
]

# Convert to year fractions
autocall_times = np.array([
    (autocall_date - issue_date).days / 365.0 for autocall_date in autocall_dates
])

# Convert to timesteps
autocall_timesteps = (autocall_times / dt).astype(int)

# Coupon payment dates (3 days after each valuation date)
coupon_dates = [val_date + timedelta(days=3) for val_date in valuation_dates]

# Convert to year fractions
coupon_times = np.array([
    (coupon_date - issue_date).days / 365.0 for coupon_date in coupon_dates
])

# Convert to timesteps
coupon_timesteps = (coupon_times / dt).astype(int)

In [5]:
# ENTER INPUT FOR: whether option is call (1) or put (0)
cp = 1

In [6]:
# Check stability condition
S_max = SU
stability_condition = dS**2 / (2 * sigma**2 * S_max**2)
dt_actual = T / imax

print(f"Stability requires dt ≤ {stability_condition:.8f}")
print(f"Actual dt = {dt_actual:.8f}")
print(f"Stable: {dt_actual <= stability_condition}")

# If unstable, increase imax
if dt_actual > stability_condition:
    imax_min = int(T / stability_condition) + 1
    print(f"Minimum imax needed for stability: {imax_min}")

Stability requires dt ≤ 0.00014364
Actual dt = 0.00013699
Stable: True


## Explicit Finite Difference Function
### When knock in triggered

In [7]:
def EXPFD_model_autocall(S0, K, T, r, q, sigma, SU, jmaxmin, jmaxmax, jmaxstep,
                imax, val_timesteps, autocall_timesteps, coupon, coupon_times, autocall_times):

    #LIST TO SAVE RESULTS
    #Assumes that SL= 0, can be relaxed, see notes for updates A, B, C values in this case

    expfd_result = []
    boundary_values_list = []

    for jmax in range(jmaxmin, jmaxmax, jmaxstep):

    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [imax+1,jmax+1] TO STORE V AT ALL STEPS
    # V[imax, jmax]

        V = np.zeros([imax+1, jmax+1])

    # CREATE ONE DIMENSIONAL ARRAY OF SIZE [jmax+1] TO STORE A, B, C VALUES AT ALL STEPS
    # A[jmax], B[jmax], C[jmax]

        A = np.zeros([jmax+1])
        B = np.zeros([jmax+1])
        C = np.zeros([jmax+1])

    # Set up time and S steps

        dt = T / imax
        dS = SU/jmax


    # CALCULATE OPTION VALUES AND PROBABILITIES
    # Start at the last step number because we are going to be moving backwards from times step imax to times step 0

        i = imax

        for j in range(0, jmax+1):
    # Then, calculate the value of the option at that exact position within the binomial tree
    # Also calculate the probabilities A, B, C
            S = j*dS
            if S >= S0:
              V[i,j] = 1000.0 + coupon
            elif S >= K:
              V[i,j] = 1000.0*S/S0 + coupon
            else:
              V[i,j] = 1000.0*S/S0
              # V[i, j] = np.maximum(j*dS - K, 0)
            A[j] = (0.5 * (sigma ** 2) * (j ** 2) + 0.5 * (r - q) * j) * dt
            B[j] =  1 - sigma ** 2 * j ** 2 * dt
            C[j] = (0.5 * (sigma ** 2) * (j ** 2) - 0.5 * (r - q) * j) * dt

    #Now go back in time
        for i in range(imax-1, -1, -1):

            #Lower boundary condition
            V[i, 0] = 0

            #regular finite difference formula
            for j in range(1, jmax, 1):
                if i in val_timesteps:
                  if j*dS >= K:
                    V[i,j] = (A[j]*V[i+1,j+1]+B[j]*V[i+1,j]+C[j]*V[i+1,j-1]) / (1+r*dt) + coupon
                  else: V[i,j] = (A[j]*V[i+1,j+1]+B[j]*V[i+1,j]+C[j]*V[i+1,j-1]) / (1+r*dt)
                elif i in autocall_timesteps:

                  if j*dS >= S0:
                    V[i,j] = 1000.0 + coupon

                  else:
                    V[i,j] = (A[j]*V[i+1,j+1]+B[j]*V[i+1,j]+C[j]*V[i+1,j-1]) / (1+r*dt)
                else:
                  V[i,j] = (A[j]*V[i+1,j+1]+B[j]*V[i+1,j]+C[j]*V[i+1,j-1]) / (1+r*dt)

            #Upper boundary condition
            if i in val_timesteps:
                # PV of all remaining coupons + PV of principal at maturity
                remaining_coupons_pv = sum([coupon * np.exp(-r*(ct - i*dt))
                               for ct in coupon_times if ct > i*dt])
                principal_pv = (1000 + coupon) * np.exp(-r*(T - i*dt))
                V[i,jmax] = remaining_coupons_pv + principal_pv

            elif i in autocall_timesteps:
                V[i,jmax] = (1000 + coupon)
            else:
                # For very high S, note will be autocalled at next opportunity
                # Discount the autocall payment to next autocall date
                next_autocall_time = None
                for ac_time in autocall_times:
                    if ac_time > i*dt:
                        next_autocall_time = ac_time
                        break

                if next_autocall_time:
                    V[i,jmax] = (1000 + coupon) * np.exp(-r*(next_autocall_time - i*dt))
                else:
                    # After last autocall date, discount to maturity
                    V[i,jmax] = (1000 + coupon) * np.exp(-r*(T - i*dt))

        # Store boundary values at S = K for all time steps
        j_K = int(K / dS)
        boundary_values_at_K = V[:, j_K].copy()  # Copy to avoid reference issues
        boundary_values_list.append(boundary_values_at_K)  # Store separately



    # RELAY OUTPUTS TO DICTIONARY
        jcritreal = S0/dS
        jcrit = int(jcritreal)
        jcritK = int(K/dS)+1
        Vcrit = V[0,jcrit]+ (S0 - jcrit * dS) / (dS) * (V[0,jcrit+1] - V[0,jcrit])
        Klambda = (jcritK*dS - K)/dS

        output = {'S_steps': jmax, 't_steps': imax, 'EXP': Vcrit, 'Lambda': Klambda}
        expfd_result.append(output)

    return expfd_result, boundary_values_list

In [8]:
jmaxmin = 250
jmaxmax = 350
jmaxstep = 10 # check this once

In [9]:
expfd, boundary_values = EXPFD_model_autocall(S0, K, T, r, q, sigma, SU,
                                              jmaxmin, jmaxmax, jmaxstep, imax,
                                              val_timesteps, autocall_timesteps, coupon, coupon_times, autocall_times)

# CREATE A DATAFRAME FROM THE BINOMIAL MODEL OUTPUT
df = pd.DataFrame.from_dict(expfd)

In [10]:
df

,S_steps,t_steps,EXP,Lambda
0,250,7920,965.895264,1.00
1,260,7920,966.006119,0.88
2,270,7920,966.107668,0.76
3,280,7920,966.201026,0.64
4,290,7920,966.287140,0.52
5,300,7920,966.366816,0.40
6,310,7920,966.440745,0.28
7,320,7920,966.509524,0.16
8,330,7920,966.573670,0.04
9,340,7920,966.043923,0.92


In [11]:
boundary_values

[array([800.35545183, 800.36005472, 800.36465675, ..., 780.33819858,
        780.17524925, 780.        ]),
 array([799.53770377, 799.54229475, 799.54688489, ..., 779.20977383,
        779.03511201, 778.84615385]),
 array([798.77881611, 798.78339607, 798.78797522, ..., 778.16761535,
        777.9809598 , 777.77777778]),
 array([798.07266421, 798.07723395, 798.08180291, ..., 777.20254809,
        777.00363512, 776.78571429]),
 array([797.4139434 , 797.41850365, 797.42306313, ..., 776.30665944,
        776.09524357, 775.86206897]),
 array([796.79803643, 796.80258783, 796.80713848, ..., 775.47308876,
        775.24894334, 775.        ]),
 array([796.22090575, 796.22544887, 796.22999127, ..., 774.69585757,
        774.45877543, 774.19354839]),
 array([795.67900538, 795.68354077, 795.68807545, ..., 773.96973165,
        773.71952571, 773.4375    ]),
 array([795.16920847, 795.1737366 , 795.17826403, ..., 773.29010814,
        773.02661208, 772.72727273]),
 array([800.33156935, 800.33628419, 8

In [12]:
# Check if boundary values are reasonable
print(f"Sample boundary values: {boundary_values[:10]}")
print(f"Boundary at t=0: {boundary_values[0]}")

Sample boundary values: [array([800.35545183, 800.36005472, 800.36465675, ..., 780.33819858,
       780.17524925, 780.        ]), array([799.53770377, 799.54229475, 799.54688489, ..., 779.20977383,
       779.03511201, 778.84615385]), array([798.77881611, 798.78339607, 798.78797522, ..., 778.16761535,
       777.9809598 , 777.77777778]), array([798.07266421, 798.07723395, 798.08180291, ..., 777.20254809,
       777.00363512, 776.78571429]), array([797.4139434 , 797.41850365, 797.42306313, ..., 776.30665944,
       776.09524357, 775.86206897]), array([796.79803643, 796.80258783, 796.80713848, ..., 775.47308876,
       775.24894334, 775.        ]), array([796.22090575, 796.22544887, 796.22999127, ..., 774.69585757,
       774.45877543, 774.19354839]), array([795.67900538, 795.68354077, 795.68807545, ..., 773.96973165,
       773.71952571, 773.4375    ]), array([795.16920847, 795.1737366 , 795.17826403, ..., 773.29010814,
       773.02661208, 772.72727273]), array([800.33156935, 800.33628

### When knock in not triggered

In [13]:
def EXPFD_model_nottriggered(S0, K, T, r, q, sigma, SU, SL, jmaxmin, jmaxmax,
                             jmaxstep,imax, val_timesteps, autocall_timesteps,
                             coupon, coupon_times, boundary_values_list):

    #LIST TO SAVE RESULTS
    #Assumes that SL= 0, can be relaxed, see notes for updates A, B, C values in this case

    expfd_result = []

    for idx, jmax in enumerate(range(jmaxmin, jmaxmax, jmaxstep)):

    # CREATE TWO DIMENSIONAL ARRAY OF SIZE [imax+1,jmax+1] TO STORE V AT ALL STEPS
    # V[imax, jmax]

        V = np.zeros([imax+1, jmax+1])

    # CREATE ONE DIMENSIONAL ARRAY OF SIZE [jmax+1] TO STORE A, B, C VALUES AT ALL STEPS
    # A[jmax], B[jmax], C[jmax]

        A = np.zeros([jmax+1])
        B = np.zeros([jmax+1])
        C = np.zeros([jmax+1])

    # Set up time and S steps

        dt = T / imax
        dS = (SU-SL)/jmax
        # Find starting index for K
        j_K = int(K / dS)

        # Get boundary values for this grid size
        boundary_values_at_K = boundary_values_list[idx]


    # CALCULATE OPTION VALUES AND PROBABILITIES
    # Start at the last step number because we are going to be moving backwards from times step imax to times step 0

        i = imax

        for j in range(j_K, jmax+1):
    # Then, calculate the value of the option at that exact position within the binomial tree
    # Also calculate the probabilities A, B, C
            S = SL + j*dS
            V[i,j] = 1000.0 + coupon
            A[j] = (0.5 * (sigma ** 2) * (j ** 2) + 0.5 * (r - q) * j) * dt
            B[j] = 1 - sigma ** 2 * j ** 2 * dt
            C[j] = (0.5 * (sigma ** 2) * (j ** 2) - 0.5 * (r - q) * j) * dt

    #Now go back in time
        for i in range(imax-1, -1, -1):

            # Lower boundary at j = j_K (S = K)
            V[i, j_K] = boundary_values_at_K[i]


            #regular finite difference formula
            for j in range(j_K+1, jmax, 1):
                if i in val_timesteps:
                  if (SL + j*dS) > K:
                    V[i,j] = (A[j]*V[i+1,j+1]+B[j]*V[i+1,j]+C[j]*V[i+1,j-1]) / (1+r*dt) + coupon
                  else: V[i,j] = (A[j]*V[i+1,j+1]+B[j]*V[i+1,j]+C[j]*V[i+1,j-1]) / (1+r*dt)
                elif i in autocall_timesteps:
                  if j*dS > S0:
                    V[i,j] = 1000.0 + coupon
                  else:
                    V[i,j] = (A[j]*V[i+1,j+1]+B[j]*V[i+1,j]+C[j]*V[i+1,j-1]) / (1+r*dt)
                else:
                  V[i,j] = (A[j]*V[i+1,j+1]+B[j]*V[i+1,j]+C[j]*V[i+1,j-1]) / (1+r*dt)

            #Upper boundary condition
            if i in val_timesteps:
                # PV of all remaining coupons + PV(1000 + last coupon)
                remaining_coupons_pv = sum([coupon * np.exp(-r*(ct - i*dt))
                                          for ct in coupon_times if ct > i*dt])
                final_payment_pv = (1000 + coupon) * np.exp(-r*(T - i*dt))
                V[i,jmax] = remaining_coupons_pv + final_payment_pv

            elif i in autocall_timesteps:
                V[i,jmax] = (1000 + coupon) * np.exp(-r*(T - i*dt))
            else:
                V[i,jmax] = 1000 + coupon

    # RELAY OUTPUTS TO DICTIONARY
        jcritreal = S0/dS
        jcrit = int(jcritreal)
        jcritK = int(K/dS)+1
        Vcrit = V[0,jcrit]+ (S0 - jcrit * dS) / (dS) * (V[0,jcrit+1] - V[0,jcrit])
        Klambda = (jcritK*dS - K)/dS
        output = {'S_steps': jmax, 't_steps': imax, 'EXP': Vcrit, 'Lambda': Klambda}
        expfd_result.append(output)

    return expfd_result

In [14]:
non_triggered_results = EXPFD_model_nottriggered(S0, K, T, r, q, sigma, SU, SL, jmaxmin, jmaxmax,
                             jmaxstep,imax, val_timesteps, autocall_timesteps,
                             coupon, coupon_times, boundary_values)

In [15]:
# CREATE A DATAFRAME FROM THE BINOMIAL MODEL OUTPUT
df_t = pd.DataFrame.from_dict(non_triggered_results)

In [16]:
df_t

,S_steps,t_steps,EXP,Lambda
0,250,7920,982.028290,1.00
1,260,7920,982.410280,0.88
2,270,7920,982.764717,0.76
3,280,7920,983.094464,0.64
4,290,7920,983.402003,0.52
5,300,7920,983.689496,0.40
6,310,7920,983.958833,0.28
7,320,7920,984.211676,0.16
8,330,7920,984.449489,0.04
9,340,7920,982.325892,0.92
